## Build BERT model

In [1]:
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow.keras.models import Model
import bert
from bert import tokenization
import numpy as np
from tqdm import tqdm
import gc
from tensorflow.keras import backend as K
import pandas as pd

/home/scheuererra68323/venv37/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
max_seq_length = 64

input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
 name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
 name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
 name="segment_ids")
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
 trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])


model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=[pooled_output, sequence_output])


# See BERT paper: https://arxiv.org/pdf/1810.04805.pdf
# And BERT implementation convert_single_example() at https://github.com/google-research/bert/blob/master/run_classifier.py

def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids


vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

## Apply BERT tokenizer

In [3]:
#df = pd.read_parquet("/mnt/md0/user/scheuererra68323/LO_SARD102/LO_SARD102_TokenAnon_wExtFuncCalls_Labeled.parquet")
df = pd.read_parquet("/mnt/md0/user/scheuererra68323/JTT/JTT_TokenAnon_wExtFuncCalls_Labeled.parquet")
df.head()

,path,line_start,line_stop,code_snippet,external_function_names,dump_tokens_output,token_anon,CWE-327,CWE-114,CWE-121,...,CWE-775,CWE-780,CWE-785,CWE-789,CWE-078,CWE-832,CWE-835,CWE-843,CWE-090,is_vulnerable
index,,,,,,,,,,,,,,,,,,,,,
83452,/mnt/md0/user/scheuererra68323/testset_jtt/C/t...,22,39,"b'[""unsigned int data;"", "" data = 0;"", "" ...","b'[""printUnsignedLine"", ""globalReturnsTrue""]'",unsigned 'unsigned'\t [StartOfLine]\tLoc=<<std...,"b'[""unsigned"", ""int"", ""identifier0"", "";"", ""ide...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
248990,/mnt/md0/user/scheuererra68323/testset_jtt/C/t...,199,203,"b'[""goodG2B();"", "" goodB2G();""]'",b'[]',identifier 'goodG2B'\t [StartOfLine]\tLoc=<<st...,"b'[""identifier0"", ""("", "")"", "";"", ""identifier1""...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
220067,/mnt/md0/user/scheuererra68323/testset_jtt/C/t...,56,67,"b'[""int64_t * data;"", "" /* Initialize data*...","b'[""CWE762_Mismatched_Memory_Management_Routin...",identifier 'int64_t'\t [StartOfLine]\tLoc=<<st...,"b'[""identifier0"", ""*"", ""identifier1"", "";"", ""id...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
283612,/mnt/md0/user/scheuererra68323/testset_jtt/C/t...,52,57,"b'[""printIntLine(data->intOne);"", "" /* POTE...",b'[]',identifier 'printIntLine'\t [StartOfLine]\tLoc...,"b'[""identifier0"", ""("", ""identifier1"", ""->"", ""i...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
142227,/mnt/md0/user/scheuererra68323/testset_jtt/C/t...,46,52,"b'[""if(GLOBAL_CONST_FIVE==5)"", "" {"", "" ...","b'[""signal""]'",if 'if'\t [StartOfLine]\tLoc=<<stdin>:1:1>\nl_...,"b'[""if"", ""("", ""identifier0"", ""=="", ""<numeric_c...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [4]:
debug = False

def bert_tokenize(token_anon):
    #print(token_anon)
    glued = " ".join(str(token_anon))
    bert_tokens = tokenizer.tokenize(glued)
    
    bert_tokens = ["[CLS]"] + bert_tokens + ["[SEP]"]
    if debug:
        print()
    
    return bert_tokens
    
tqdm.pandas()

df['bert_tokens'] = df.token_anon.progress_apply(bert_tokenize)

print(df.shape)
df.head()

/home/scheuererra68323/venv37/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 24999/24999 [02:08<00:00, 195.27it/s]

(24999, 128)


,path,line_start,line_stop,code_snippet,external_function_names,dump_tokens_output,token_anon,CWE-327,CWE-114,CWE-121,...,CWE-780,CWE-785,CWE-789,CWE-078,CWE-832,CWE-835,CWE-843,CWE-090,is_vulnerable,bert_tokens
index,,,,,,,,,,,,,,,,,,,,,
83452,/mnt/md0/user/scheuererra68323/testset_jtt/C/t...,22,39,"b'[""unsigned int data;"", "" data = 0;"", "" ...","b'[""printUnsignedLine"", ""globalReturnsTrue""]'",unsigned 'unsigned'\t [StartOfLine]\tLoc=<<std...,"b'[""unsigned"", ""int"", ""identifier0"", "";"", ""ide...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,"[[CLS], b, ', [, "", u, n, s, i, g, n, e, d, "",..."
248990,/mnt/md0/user/scheuererra68323/testset_jtt/C/t...,199,203,"b'[""goodG2B();"", "" goodB2G();""]'",b'[]',identifier 'goodG2B'\t [StartOfLine]\tLoc=<<st...,"b'[""identifier0"", ""("", "")"", "";"", ""identifier1""...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,"[[CLS], b, ', [, "", i, d, e, n, t, i, f, i, e,..."
220067,/mnt/md0/user/scheuererra68323/testset_jtt/C/t...,56,67,"b'[""int64_t * data;"", "" /* Initialize data*...","b'[""CWE762_Mismatched_Memory_Management_Routin...",identifier 'int64_t'\t [StartOfLine]\tLoc=<<st...,"b'[""identifier0"", ""*"", ""identifier1"", "";"", ""id...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,"[[CLS], b, ', [, "", i, d, e, n, t, i, f, i, e,..."
283612,/mnt/md0/user/scheuererra68323/testset_jtt/C/t...,52,57,"b'[""printIntLine(data->intOne);"", "" /* POTE...",b'[]',identifier 'printIntLine'\t [StartOfLine]\tLoc...,"b'[""identifier0"", ""("", ""identifier1"", ""->"", ""i...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,"[[CLS], b, ', [, "", i, d, e, n, t, i, f, i, e,..."
142227,/mnt/md0/user/scheuererra68323/testset_jtt/C/t...,46,52,"b'[""if(GLOBAL_CONST_FIVE==5)"", "" {"", "" ...","b'[""signal""]'",if 'if'\t [StartOfLine]\tLoc=<<stdin>:1:1>\nl_...,"b'[""if"", ""("", ""identifier0"", ""=="", ""<numeric_c...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,"[[CLS], b, ', [, "", i, f, "", ,, "", (, "", ,, "",..."


In [5]:
bert_tokens = df
bert_tokens.bert_tokens.map(len).describe()

count     24999.000000
mean        477.371935
std        1972.992647
min          10.000000
25%          91.000000
50%         308.000000
75%         570.000000
max      141331.000000
Name: bert_tokens, dtype: float64

In [6]:
bert_tokens = bert_tokens.loc[bert_tokens.bert_tokens.map(len) <= max_seq_length]
bert_tokens.bert_tokens.map(len).describe()

count    5607.000000
mean       45.213126
std        12.360028
min        10.000000
25%        32.000000
50%        46.000000
75%        58.000000
max        64.000000
Name: bert_tokens, dtype: float64

## Predict BERT embeddings

In [7]:
def bert_embed(bert_tokens):
    input_ids = get_ids(bert_tokens, tokenizer, max_seq_length)
    input_masks = get_masks(bert_tokens, max_seq_length)
    input_segments = get_segments(bert_tokens, max_seq_length)

    pooled_output, sequence_output = model.predict_on_batch(
        [[input_ids],[input_masks],[input_segments]])
    #K.clear_session()
    return sequence_output.squeeze()

tqdm.pandas()
bert_embeddings = bert_tokens.bert_tokens.progress_apply(bert_embed)

/home/scheuererra68323/venv37/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 5607/5607 [07:37<00:00, 12.26it/s]


In [8]:
bert_embeddings.map(len).describe()

count    5607.0
mean       64.0
std         0.0
min        64.0
25%        64.0
50%        64.0
75%        64.0
max        64.0
Name: bert_tokens, dtype: float64

## Save arrays into files

In [9]:
X = np.stack(bert_embeddings.values)
print(X.shape)

(5607, 64, 768)


In [10]:
y = bert_tokens.drop(["code_snippet", "external_function_names", "dump_tokens_output",
         "path" ,"line_start" ,"line_stop" ,"token_anon", 'bert_tokens'], axis=1)
y = y.where(pd.notnull(y), 0).astype(int)
print(y.shape)
print(y.dtypes)

(5607, 120)
CWE-327          int64
CWE-114          int64
CWE-121          int64
CWE-135          int64
CWE-126          int64
                 ...  
CWE-832          int64
CWE-835          int64
CWE-843          int64
CWE-090          int64
is_vulnerable    int64
Length: 120, dtype: object


In [11]:
# shuffle data before saving
#rng_state = np.random.get_state()
#np.random.shuffle(X)
#np.random.set_state(rng_state)
#np.random.shuffle(y)

In [13]:
#np.save('LOSARD102_bert64_X.npy', X)
np.save('JTT_bert64_X.npy', X)

In [14]:
#y.to_hdf('LOSARD102_bert64_y.h5', key='y')
y.to_hdf('JTT_bert64_y.h5', key='y')